R. N. Modification of the original code from Prof Avi Rosenfeld.

Note: 2 queens and 3 queens do not have a solution.

This is the notebook version of the code. I will use this to explain the homework.  I used parts of the code from: https://www.sanfoundry.com/python-program-solve-n-queen-problem-without-recursion/

As we did in class, we will represent the board as a one-dimensional array where each position  in the arrray is the row value. So if the array is: [1, 3, 0, 2], then the  queen in the first row is in column 1 (columns are labeled from 0--3), the queen in the second row is in column 3 (the last column), the queen in the third row is in column 0 and the  queen in the last row is in column 2.

Let's setup one iteration of the British Museum algorithm-- we'll put down n=8 queens randomly.

In [42]:
import random
def place_n_queens(n):
    assert n >= 0
    columns = [random.randrange(0,n) for x in range(n)]
    return columns


def displayBoard(columns):
    if not columns:
        return
    n = len(columns)
    print(columns)
    for rowVal in range(n):
        rowStr = ["." for x in range(n)]
        rowStr[columns[rowVal]] = '♛'
        print("  ".join(rowStr))
    print()


In [43]:
columns = place_n_queens(8)
displayBoard(columns)

[3, 2, 6, 4, 5, 3, 1, 2]
.  .  .  ♛  .  .  .  .
.  .  ♛  .  .  .  .  .
.  .  .  .  .  .  ♛  .
.  .  .  .  ♛  .  .  .
.  .  .  .  .  ♛  .  .
.  .  .  ♛  .  .  .  .
.  ♛  .  .  .  .  .  .
.  .  ♛  .  .  .  .  .



This arrangement is of course is not necessary a solution, so we'll iterate this till we get a solution.  This is the British Museum algorithm.


We first introduce a general purpose harness for running the different algorithm, with the signature:

```
columns, num_iterations, number_moves, converged=solve_algorithm(size)```

In [57]:
def solve_nqueens(solve_algorithm, size = 8,  ntrials = 1):

  num_iterations_list, num_moves_list, convergence_list = [], [], []
  for i in range(0, ntrials):
    num_iterations = 0
    num_moves = 0
    columns, num_iterations, num_moves, converged=solve_algorithm(size)
    num_iterations_list.append(num_iterations)
    num_moves_list.append(num_moves)
    convergence_list.append(converged)
    print(f"is converged: {converged}")
    print(f"is valid solution obtained: {is_valid_solution(columns)}")
    print(f"number of iterations: {num_iterations}")
    print(f"number of moves: {num_moves}")
    if converged:
        print("display the results")
        displayBoard(columns)

  if ntrials > 1:
    print(f"average number of iterations: {sum(num_iterations_list)/len(num_iterations_list)}")
    print(f"average number of moves: {sum(num_moves_list)/len(num_moves_list)}")
    print(f"average convergence: {sum(convergence_list)/len(convergence_list)}")

Now what?  Can you implement the British Museum Algorithm?  How many moves and iterations did it take to solve the 4 queens problem?  

How many moves/iterations did it take to solve the 8 queens (if at all)?

In [58]:

def solve_britishmuseum(size):

  MAX_NUMBER_OF_ITERATIONS = 1000000
  number_of_iterations, number_of_moves, converged = 0, 0, False

  while not converged and number_of_iterations < MAX_NUMBER_OF_ITERATIONS:
    number_of_iterations += 1
    columns = place_n_queens(size)
    number_of_moves += 8
    if is_valid_solution(columns):
      converged = True

  return columns, number_of_iterations, number_of_moves, converged

In [59]:

solve_nqueens(solve_britishmuseum, size = 8,  ntrials = 1)


is converged: True
is valid solution obtained: True
number of iterations: 8882
number of moves: 71056
display the results
[1, 4, 6, 0, 2, 7, 5, 3]
.  ♛  .  .  .  .  .  .
.  .  .  .  ♛  .  .  .
.  .  .  .  .  .  ♛  .
♛  .  .  .  .  .  .  .
.  .  ♛  .  .  .  .  .
.  .  .  .  .  .  .  ♛
.  .  .  .  .  ♛  .  .
.  .  .  ♛  .  .  .  .



This approach is not very efficient, so we'll write a simple DFS search with backtracking:

First some helper functions ..


1. place_next_row

2. remove_current_row

3. is_col_place_in_next_row_valid

In [48]:
def place_next_row(col, columns):
    columns.append(col)

def remove_current_row(columns):
    if len(columns) > 0:
        return columns.pop()
    return -1

def is_col_place_in_next_row_valid(col, columns):
    # new row
    row = len(columns)

    def check_no_conflicts(col, row, queen_col, queen_row):
      return (col != queen_col) and (queen_col - queen_row != col - row) and (queen_col + queen_row != col + row)

    # return True if all  existing queens do not conflict with the proposed (row, col) position
    return all([check_no_conflicts(col, row, queen_col, queen_row) for queen_row, queen_col in enumerate(columns)])

def is_valid_solution(columns):
    return all([is_col_place_in_next_row_valid(col, columns[:row]) for row, col in enumerate(columns)])

We can now write the desired dfs code by starting with a blank board and placing the queens in one row at a time, starting with the first row.



In [49]:
def solve_dfs(size):
    columns = []
    number_of_moves = 0 #where do I change this so it counts the number of Queen moves?
    number_of_iterations = 0
    row = 0 # current row
    col = 0 # always start each row at leftmost col
    # iterate over rows of board
    while True:
        #place queen in next row
        # print("I have ", row, " number of queens put down")
        # print(columns)
        # print(f"number_of_moves: {number_of_moves}")
        while col < size:
            number_of_iterations += 1
            if is_col_place_in_next_row_valid(col, columns):
                place_next_row(col, columns)
                number_of_moves += 1
                # print(f"placed: row: {row}, col: {col}")
                row += 1
                col = 0
                break
            else:
                # print(f"not placed: row: {row}, col: {col}")
                col += 1


        # could not find an open col in this row (backtrack),  or board is full
        if (col == size or row == size):
            number_of_iterations+=1
            # if board is full, we have a solution
            if row == size:
                print("I did it! Here is my solution")
                display(columns)
                converged = True
                return columns, number_of_iterations, number_of_moves, converged
            # else couldn't find a solution so need to backtrack
            # print("start to backtrack ... ")
            prev_col = remove_current_row(columns)
            if (prev_col == -1): # backtracked past column 1
                print("There are no solutions")
                #print(number_of_moves)
                converged = False
                return columns, number_of_iterations, number_of_moves, converged
            # retry previous row again
            row -= 1
            # start to now check at col = (1 + value of prev_column in the row)
            col = 1 + prev_col



Does the DFS algorithm get to 30 (not easily at all!).  To show this check the values for num_iterations and number_moves for values of 10, 20 and 30.  Feel free to stop the DFS algorithm after 10,000,000 iterations!


In [50]:
solve_nqueens(solve_dfs, size = 8,  ntrials = 1)

I did it! Here is my solution


[0, 4, 7, 5, 2, 6, 1, 3]

is converged: True
is valid solution obtained: True
number of iterations: 982
number of moves: 113
display the results
[0, 4, 7, 5, 2, 6, 1, 3]
♛  .  .  .  .  .  .  .
.  .  .  .  ♛  .  .  .
.  .  .  .  .  .  .  ♛
.  .  .  .  .  ♛  .  .
.  .  ♛  .  .  .  .  .
.  .  .  .  .  .  ♛  .
.  ♛  .  .  .  .  .  .
.  .  .  ♛  .  .  .  .



Now implement your Hill Climbing Heuristic Repair Algorithm and check the values for num_iterations and number_moves for values of 10, 20 and 30.

First some useful helper functions

In [51]:
def count_row_conflicts(row, columns):
  assert row < len(columns)
  row_conflicts = 0
  col = columns[row]
  for queen_row, queen_col in enumerate(columns):
    if queen_row != row:
      row_conflicts +=  (col == queen_col) + (queen_col - queen_row == col - row) + (queen_col + queen_row == col + row)
  return row_conflicts

def count_conflicts(columns):
  return sum([count_row_conflicts(row, columns) for row in range(len(columns))])/2

# some testing code
def test_count_row_conflicts():
  print("display a board with no conflicts")
  columns = [5, 1, 6, 0, 2, 4, 7, 3] # solution
  displayBoard(columns)
  for row in range(len(columns)):
    print(f"row: {row}, row conflicts: {count_row_conflicts(row, columns)}")
  assert all([count_row_conflicts(row, columns) == 0 for row in range(len(columns))])

  print()
  print("display a board with conflicts")
  columns = [5, 1, 0, 6, 2, 4, 7, 3] # not a solution

  displayBoard(columns)
  for row in range(len(columns)):
    print(f"row: {row}, row conflicts: {count_row_conflicts(row, columns)}")
  assert not all([count_row_conflicts(row, columns) == 0 for row in range(len(columns))])


def test_count_conflicts():

  columns = [5, 1, 6, 0, 2, 4, 7, 3] # solution
  assert count_conflicts(columns) == 0

  print()
  columns = [5, 1, 0, 6, 2, 4, 7, 3] # not a solution
  assert count_conflicts(columns) == 3


test_count_row_conflicts()
test_count_conflicts()


display a board with no conflicts
[5, 1, 6, 0, 2, 4, 7, 3]
.  .  .  .  .  ♛  .  .
.  ♛  .  .  .  .  .  .
.  .  .  .  .  .  ♛  .
♛  .  .  .  .  .  .  .
.  .  ♛  .  .  .  .  .
.  .  .  .  ♛  .  .  .
.  .  .  .  .  .  .  ♛
.  .  .  ♛  .  .  .  .

row: 0, row conflicts: 0
row: 1, row conflicts: 0
row: 2, row conflicts: 0
row: 3, row conflicts: 0
row: 4, row conflicts: 0
row: 5, row conflicts: 0
row: 6, row conflicts: 0
row: 7, row conflicts: 0

display a board with conflicts
[5, 1, 0, 6, 2, 4, 7, 3]
.  .  .  .  .  ♛  .  .
.  ♛  .  .  .  .  .  .
♛  .  .  .  .  .  .  .
.  .  .  .  .  .  ♛  .
.  .  ♛  .  .  .  .  .
.  .  .  .  ♛  .  .  .
.  .  .  .  .  .  .  ♛
.  .  .  ♛  .  .  .  .

row: 0, row conflicts: 0
row: 1, row conflicts: 1
row: 2, row conflicts: 2
row: 3, row conflicts: 1
row: 4, row conflicts: 1
row: 5, row conflicts: 1
row: 6, row conflicts: 0
row: 7, row conflicts: 0



In [54]:

def solve_hillclimbing(size):

  MAX_NUMBER_OF_ITERATIONS = 1000
  MAX_ITERATIONS_WITHOUT_IMPROVEMENT = 10
  number_of_iterations, number_of_moves, converged = 0, 0, False

  columns = place_n_queens(size)
  min_conflict_count = count_conflicts(columns)
  print(f"initial conflict count: {min_conflict_count}")

  while not converged and number_of_iterations < MAX_NUMBER_OF_ITERATIONS:
    number_of_iterations += 1

    #check all possible moves and pick the best
    best_row, best_col = -1, -1
    for row in range(size):
      col = columns[row]
      for new_col in range(size):
        columns[row] = new_col # swap to new state
        conflict_count = count_conflicts(columns)
        if conflict_count < min_conflict_count:
          min_conflict_count = conflict_count
          best_row, best_col = row, new_col
          improvement_count = 0  # reset counter tracking no improvement per iteration

        columns[row] = col  # swap back to old state and continue

    # replace with best solution at the end of the scan
    # print(f"min_conflict_count: {min_conflict_count}")
    if best_row != -1: # better solution found
      columns[best_row] = best_col
      number_of_moves += 1
    else:  # no better solution found
      improvement_count += 1
      if improvement_count > MAX_ITERATIONS_WITHOUT_IMPROVEMENT: # restart with another random configuration
          columns = place_n_queens(size)
          min_conflict_count = count_conflicts(columns)

    converged = min_conflict_count == 0

  return columns, number_of_iterations, number_of_moves, converged

In [55]:

solve_nqueens(solve_hillclimbing, size = 8,  ntrials = 1)

initial conflict count: 5.0
is converged: True
is valid solution obtained: True
number of iterations: 102
number of moves: 25
display the results
[1, 7, 5, 0, 2, 4, 6, 3]
.  ♛  .  .  .  .  .  .
.  .  .  .  .  .  .  ♛
.  .  .  .  .  ♛  .  .
♛  .  .  .  .  .  .  .
.  .  ♛  .  .  .  .  .
.  .  .  .  ♛  .  .  .
.  .  .  .  .  .  ♛  .
.  .  .  ♛  .  .  .  .



Now implement your Forward Checking and check the values for num_iterations and number_moves for values of 10, 20 and 30. As this algorithm is not deterministic, run the algorithm 30 times for each of the values for n and report on the average values for the two counters.

In [75]:
def displayConstraints(domains):
    if not columns:
        return
    n = len(domains)
    print(f"size: {n}")
    for domain in domains:
        rowStr = ["." if x in domain else "X" for x in range(n)]
        print("  ".join(rowStr))
    print()


def is_col_open_in_row(row, col, domains):
  return col in domains[row]

def forward_check(row, col, domains):
  for r, domain in enumerate(domains[row:], row):
    domain.discard(col) # remove same column
    domain.discard(abs(col-row+r)) # remove diagonal
    domain.discard(abs(col+row-r)) # remove anti diagonal

def forward_check_redo(size, columns):
  domains = [set(range(size)) for _ in range(size)]
  for row, col in enumerate(columns):
    for r, domain in enumerate(domains[row:], row):
      domain.discard(col) # remove same column
      domain.discard(abs(col-row+r)) # remove diagonal
      domain.discard(abs(col+row-r)) # remove anti diagonal
  return domains

def initialize_domains(size):
  assert size >= 0
  domains = [set(range(size)) for x in range(size)]
  return domains

domains = initialize_domains(5)  # Initialize domains with all columns
print(domains)

print("initial board")
displayConstraints(domains)

forward_check(0, 0, domains)
print("board after (0,0)")
displayConstraints(domains)

forward_check(1, 3, domains)
print("board after (1,3)")
displayConstraints(domains)

columns = [0,3]
domains = forward_check_redo(size,columns)
print("board after redo for (0,0), (1,3)")
displayConstraints(domains)


[{0, 1, 2, 3, 4}, {0, 1, 2, 3, 4}, {0, 1, 2, 3, 4}, {0, 1, 2, 3, 4}, {0, 1, 2, 3, 4}]
initial board
size: 5
.  .  .  .  .
.  .  .  .  .
.  .  .  .  .
.  .  .  .  .
.  .  .  .  .

board after (0,0)
size: 5
X  .  .  .  .
X  X  .  .  .
X  .  X  .  .
X  .  .  X  .
X  .  .  .  X

board after (1,3)
size: 5
X  .  .  .  .
X  X  .  X  .
X  .  X  X  X
X  X  .  X  .
X  .  .  X  X

board after redo for (0,0), (1,3)
size: 8
X  .  .  .  .  .  .  .
X  X  .  X  .  .  .  .
X  .  X  X  X  .  .  .
X  X  .  X  .  X  .  .
X  .  .  X  X  .  X  .
X  X  .  X  .  X  .  X
X  .  X  X  .  .  X  .
X  .  .  X  .  .  .  X



In [76]:
def solve_forward_checking(size):
    columns = []
    domains = [set(range(size)) for _ in range(size)]  # Initialize domains with all columns
    number_of_moves = 0 #where do I change this so it counts the number of Queen moves?
    number_of_iterations = 0

    row = 0 # current row
    col = 0 # always start each row at leftmost col
    # iterate over rows of board
    while True:
        while col < size:
            number_of_iterations += 1
            if col in domains[row]:
                place_next_row(col, columns)
                forward_check(row, col, domains)
                number_of_moves += 1
                print(f"placed: row: {row}, col: {col}")
                row += 1
                col = 0
                break
            else:
                # print(f"not placed: row: {row}, col: {col}")
                col += 1

        # could not find an open col in this row (backtrack),  or board is full
        if (col == size or row == size):
            number_of_iterations+=1
            # if board is full, we have a solution
            if row == size:
                print("I did it! Here is my solution")
                display(columns)
                converged = True
                return columns, number_of_iterations, number_of_moves, converged
            # else couldn't find a solution so need to backtrack
            # print("start to backtrack ... ")
            prev_col = remove_current_row(columns)
            if (prev_col == -1): # backtracked past column 1
                print("There are no solutions")
                #print(number_of_moves)
                converged = False
                return columns, number_of_iterations, number_of_moves, converged
            # retry previous row again, redo the forward checking
            row -= 1
            domains = forward_check_redo(size,columns)
            # start to now check at col = (1 + value of prev_column in the row)
            col = 1 + prev_col



In [77]:
solve_nqueens(solve_forward_checking, size = 8,  ntrials = 1)

placed: row: 0, col: 0
size: 8
X  .  .  .  .  .  .  .
X  X  .  .  .  .  .  .
X  .  X  .  .  .  .  .
X  .  .  X  .  .  .  .
X  .  .  .  X  .  .  .
X  .  .  .  .  X  .  .
X  .  .  .  .  .  X  .
X  .  .  .  .  .  .  X

placed: row: 1, col: 2
size: 8
X  .  .  .  .  .  .  .
X  X  X  .  .  .  .  .
X  X  X  X  .  .  .  .
X  .  X  X  X  .  .  .
X  X  X  .  X  X  .  .
X  .  X  .  .  X  X  .
X  .  X  X  .  .  X  X
X  .  X  .  X  .  .  X

placed: row: 2, col: 4
size: 8
X  .  .  .  .  .  .  .
X  X  X  .  .  .  .  .
X  X  X  X  X  .  .  .
X  .  X  X  X  X  .  .
X  X  X  .  X  X  X  .
X  X  X  .  X  X  X  X
X  .  X  X  X  .  X  X
X  X  X  .  X  .  .  X

placed: row: 3, col: 1
size: 8
X  .  .  .  .  .  .  .
X  X  X  .  .  .  .  .
X  X  X  X  X  .  .  .
X  X  X  X  X  X  .  .
X  X  X  .  X  X  X  .
X  X  X  X  X  X  X  X
X  X  X  X  X  .  X  X
X  X  X  X  X  X  .  X

placed: row: 4, col: 3
size: 8
X  .  .  .  .  .  .  .
X  X  X  .  .  .  .  .
X  X  X  X  X  .  .  .
X  X  X  X  X  X  .  .
X  X  X  X  X

[1, 3, 5, 7, 2, 0, 6, 4]

is converged: True
is valid solution obtained: True
number of iterations: 1882
number of moves: 213
display the results
[1, 3, 5, 7, 2, 0, 6, 4]
.  ♛  .  .  .  .  .  .
.  .  .  ♛  .  .  .  .
.  .  .  .  .  ♛  .  .
.  .  .  .  .  .  .  ♛
.  .  ♛  .  .  .  .  .
♛  .  .  .  .  .  .  .
.  .  .  .  .  .  ♛  .
.  .  .  .  ♛  .  .  .



Challenge Question: Write a loop solving and printing each of the n-Queens problems to 40.  Can you get either of the solutions to get all the way to 40?  Did you need to add clever tricks to make it happen?